## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from `data/wikipedia_data.html` and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in `data/nfl.csv`), MLB (baseball, in `data/mlb.csv`), NBA (basketball, in `data/nba.csv` or NHL (hockey, in `data/nhl.csv`). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with pearsonr, so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes
1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the `NHL` using `2018` data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities = pd.read_html("data/wikipedia_data.html")[1]
cities.head()

,Metropolitan area,Country,Pop. rank,Population (2016 est.)[8],B4,NFL,MLB,NBA,NHL,B6,MLS,CFL
0,New York City,United States,1,20153634,9,Giants Jets[note 1],Yankees Mets[note 2],Knicks Nets,Rangers Islanders Devils[note 3],11,Red Bulls New York City FC,—
1,Los Angeles,United States,2,13310447,8,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks,10,Galaxy Los Angeles FC[note 5],—
2,San Francisco Bay Area,United States,6,6657982,6,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7],7,Earthquakes,—
3,Chicago,United States,3,9512999,5,Bears[note 8],Cubs White Sox,Bulls[note 9],Blackhawks,6,Fire,—
4,Dallas–Fort Worth,United States,4,7233323,4,Cowboys,Rangers,Mavericks,Stars,5,FC Dallas,—


In [2]:
colunm = ['Metropolitan area', 'Population (2016 est.)[8]', 'NFL', 'MLB', 'NBA', 'NHL']
cities = cities[colunm]
cities.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True)
cities.head()

,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,New York City,20153634,Giants Jets[note 1],Yankees Mets[note 2],Knicks Nets,Rangers Islanders Devils[note 3]
1,Los Angeles,13310447,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks
2,San Francisco Bay Area,6657982,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],Cubs White Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [3]:
# remove [note ] in NFL, MLB, NBA, NHL
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")
cities.head()

C:\Users\minhh\AppData\Local\Temp\ipykernel_16620\4239142914.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
C:\Users\minhh\AppData\Local\Temp\ipykernel_16620\4239142914.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
C:\Users\minhh\AppData\Local\Temp\ipykernel_16620\4239142914.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
C:\Users\minhh\AppData\Local\Temp\ipykernel_16620\4239142914.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")


,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,New York City,20153634,Giants Jets,Yankees Mets,Knicks Nets,Rangers Islanders Devils
1,Los Angeles,13310447,Rams Chargers,Dodgers Angels,Lakers Clippers,Kings Ducks
2,San Francisco Bay Area,6657982,49ers Raiders,Giants Athletics,Warriors,Sharks
3,Chicago,9512999,Bears,Cubs White Sox,Bulls,Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [4]:
# Extract team NHL
team_NHL = cities['NHL'].apply(lambda x: x.split(' ')[0])
team_NHL.unique()

array(['Rangers', 'Kings', 'Sharks', 'Blackhawks', 'Stars', 'Capitals',
       'Flyers', 'Bruins', 'Wild', 'Avalanche', 'Panthers', 'Coyotes',
       'Red', 'Maple', '—', '', 'Lightning', 'Penguins', 'Blues',
       'Predators', 'Sabres', 'Canadiens', 'Canucks', 'Blue', 'Flames',
       'Senators', 'Oilers', 'Jets', 'Golden', 'Hurricanes', '31'],
      dtype=object)

In [5]:
# Copy two column Metropolitan area and Population from cities to team_NHL
selected_column = cities[['Metropolitan area', 'Population']].copy()
team_NHL = pd.concat([team_NHL, selected_column], axis=1)
team_NHL = team_NHL.rename(columns={'NHL': 'team'})

In [6]:
# drop "_" and "" in NHL
team_NHL['team'] = team_NHL['team'].astype(str).replace("", np.nan).replace("—", np.nan)
team_NHL['Population'] = team_NHL['Population'].astype(str).replace('–', np.nan)
team_NHL.dropna(how='any', inplace=True)

In [7]:
team_NHL = team_NHL.astype({'Metropolitan area': str, 'team': str, 'Population': int})
team_NHL.head(10)

,team,Metropolitan area,Population
0,Rangers,New York City,20153634
1,Kings,Los Angeles,13310447
2,Sharks,San Francisco Bay Area,6657982
3,Blackhawks,Chicago,9512999
4,Stars,Dallas–Fort Worth,7233323
5,Capitals,"Washington, D.C.",6131977
6,Flyers,Philadelphia,6070500
7,Bruins,Boston,4794447
8,Wild,Minneapolis–Saint Paul,3551036
9,Avalanche,Denver,2853077


In [8]:
df_NHL = pd.read_csv("data/nhl.csv")
df_NHL = df_NHL[df_NHL['year']==2018]
df_NHL.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL


In [9]:
# remove * in team and sellect column W and L
df_NHL['team'] = df_NHL['team'].str.replace(r'\*',"")
df_NHL = df_NHL[['team', 'W', 'L']]
df_NHL.head()

C:\Users\minhh\AppData\Local\Temp\ipykernel_16620\78796789.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_NHL['team'] = df_NHL['team'].str.replace(r'\*',"")


,team,W,L
0,Atlantic Division,Atlantic Division,Atlantic Division
1,Tampa Bay Lightning,54,23
2,Boston Bruins,50,20
3,Toronto Maple Leafs,49,26
4,Florida Panthers,44,30


In [10]:
# drop team have W and L the same with team like 'Atlantic Division'
indexTeam = df_NHL[ (df_NHL['team'] == df_NHL['W']) & (df_NHL['W'] == df_NHL['L']) ].index
df_NHL.drop(indexTeam, inplace=True)
df_NHL.head(10)

,team,W,L
1,Tampa Bay Lightning,54,23
2,Boston Bruins,50,20
3,Toronto Maple Leafs,49,26
4,Florida Panthers,44,30
5,Detroit Red Wings,30,39
6,Montreal Canadiens,29,40
7,Ottawa Senators,28,43
8,Buffalo Sabres,25,45
10,Washington Capitals,49,26
11,Pittsburgh Penguins,47,29


In [11]:
df_NHL['team'] = df_NHL['team'].str.replace('[\w.]* ','')
df_NHL = df_NHL.astype({'team': str,'W': int, 'L': int})
df_NHL.head()

C:\Users\minhh\AppData\Local\Temp\ipykernel_16620\1137580931.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_NHL['team'] = df_NHL['team'].str.replace('[\w.]* ','')


,team,W,L
1,Lightning,54,23
2,Bruins,50,20
3,Leafs,49,26
4,Panthers,44,30
5,Wings,30,39


In [12]:
df_NHL['W/L%'] = df_NHL['W']/(df_NHL['W']+df_NHL['L'])
df_NHL.head(10)

,team,W,L,W/L%
1,Lightning,54,23,0.701299
2,Bruins,50,20,0.714286
3,Leafs,49,26,0.653333
4,Panthers,44,30,0.594595
5,Wings,30,39,0.434783
6,Canadiens,29,40,0.420290
7,Senators,28,43,0.394366
8,Sabres,25,45,0.357143
10,Capitals,49,26,0.653333
11,Penguins,47,29,0.618421


In [13]:
merge = pd.merge(team_NHL, df_NHL, how='outer', on='team')
merge.head(10)

,team,Metropolitan area,Population,W,L,W/L%
0,Rangers,New York City,20153634.0,34.0,39.0,0.465753
1,Kings,Los Angeles,13310447.0,45.0,29.0,0.608108
2,Sharks,San Francisco Bay Area,6657982.0,45.0,27.0,0.625000
3,Blackhawks,Chicago,9512999.0,33.0,39.0,0.458333
4,Stars,Dallas–Fort Worth,7233323.0,42.0,32.0,0.567568
5,Capitals,"Washington, D.C.",6131977.0,49.0,26.0,0.653333
6,Flyers,Philadelphia,6070500.0,42.0,26.0,0.617647
7,Bruins,Boston,4794447.0,50.0,20.0,0.714286
8,Wild,Minneapolis–Saint Paul,3551036.0,45.0,26.0,0.633803
9,Avalanche,Denver,2853077.0,43.0,30.0,0.589041


In [15]:
merge = merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
merge.head()

,W/L%,Population
Metropolitan area,,
Boston,0.714286,4794447.0
Buffalo,0.357143,1132804.0
Calgary,0.513889,1392609.0
Chicago,0.458333,9512999.0
Columbus,NaN,2041520.0
